**Task 2: Automated Hyperparameter Search**

In [ ]:
from sklearn.datasets import load_iris

#step1: load the data
# Load the Iris dataset
iris = load_iris()

In [ ]:
print(iris)

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn import datasets
import numpy as np

X = iris.data  # Feature matrix (4 features)
y = iris.target  # Class labels (0, 1, 2)

print("X matrix :", X.shape)
print("y matrix:", y.shape)

# Step 2: One-hot encode labels
y = y.reshape(-1, 1)  # Reshape to column vector for encoding
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Step 3: Split the dataset
# First, split into training (70%) and temporary (30%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y_encoded, test_size=0.3, random_state=42, stratify=y)

# Further split temp set into validation (10%) and testing (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=2/3, random_state=42, stratify=y_temp)

# Step 4: Normalize feature values to [0,1] using MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)  # Fit and transform training data
X_val = scaler.transform(X_val)  # Transform validation data
X_test = scaler.transform(X_test)  # Transform test data

# Print dataset shapes
print("\nDataset Splits:")
print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Testing set: {X_test.shape}, {y_test.shape}")


X matrix : (150, 4)
y matrix: (150,)

Dataset Splits:
Training set: (105, 4), (105, 3)
Validation set: (15, 4), (15, 3)
Testing set: (30, 4), (30, 3)


In [ ]:
!pip install autogluon

In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# Convert dataset to DataFrame
X_train_df = pd.DataFrame(X_train, columns=iris.feature_names)
X_train_df['label'] = np.argmax(y_train, axis=1)  # Convert one-hot to class labels

X_test_df = pd.DataFrame(X_test, columns=iris.feature_names)
X_test_df['label'] = np.argmax(y_test, axis=1)

Use the Grid Search over the parameters defined above, Random Search, and Hyperband + Bayesian Optimization hyperparameter to search for the hyperparameters defined in Task 1.

Create a table (Each row with a configuration and column with Accuracy and F1) for Grid, Random, Hyperband, and Bayesian search and compare their accuracy and F1.


**Random search and bayesian optimization**

In [ ]:
from autogluon.common import space as ag_space
from autogluon.tabular import TabularPredictor
import pandas as pd
from sklearn.metrics import f1_score


# Define correct hyperparameter space
hyperparameters = {
    "GBM": {
        "num_boost_round": ag_space.Categorical(50, 100, 200),
        "learning_rate": ag_space.Categorical(1e-3, 1e-5),
    },
    "NN_TORCH": {
        "num_epochs": ag_space.Categorical(1, 3, 5),
        "learning_rate": ag_space.Categorical(1e-3, 1e-5),
        "batch_size": ag_space.Categorical(2, 4),
    }
}

# Train using Random Search
random_results = TabularPredictor(label="label", eval_metric="accuracy").fit(
    X_train_df,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs={"searcher": "random", "scheduler": "local", "num_trials": 10}
)

# Train using Bayesian Optimization
bayesian_results = TabularPredictor(label="label", eval_metric="accuracy").fit(
    X_train_df,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs={"searcher": "bayes", "scheduler": "local", "num_trials": 10}
)

# Get Predictions
random_preds = random_results.predict(X_test_df)
bayesian_preds = bayesian_results.predict(X_test_df)

# Get ground truth labels
y_test_true = X_test_df["label"]

# Compute accuracy
random_acc = random_results.evaluate(X_test_df)["accuracy"]
bayesian_acc = bayesian_results.evaluate(X_test_df)["accuracy"]

# Compute F1-score manually
random_f1 = f1_score(y_test_true, random_preds, average="weighted")
bayesian_f1 = f1_score(y_test_true, bayesian_preds, average="weighted")

# Store results in a DataFrame
results = [
    {"Method": "Random Search", "Accuracy": random_acc, "F1-Score": random_f1},
    {"Method": "Bayesian Optimization", "Accuracy": bayesian_acc, "F1-Score": bayesian_f1},

]

df_results = pd.DataFrame(results)
print(df_results)


No path specified. Models will be saved in: "AutogluonModels/ag-20250226_175600"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.33 GB / 12.67 GB (89.4%)
Disk Space Avail:   76.13 GB / 112.64 GB (67.6%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions 

  0%|          | 0/10 [00:00<?, ?it/s]

Fitted model: LightGBM/T1 ...
	0.9524	 = Validation score   (accuracy)
	7.81s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.9524	 = Validation score   (accuracy)
	0.41s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.9524	 = Validation score   (accuracy)
	0.64s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.8095	 = Validation score   (accuracy)
	0.6s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.9524	 = Validation score   (accuracy)
	0.57s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T6 ...
	0.9524	 = Validation score   (accuracy)
	0.29s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T7 ...
	0.9524	 = Validation score   (accuracy)
	0.38s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T8 ...
	0.3333	 = Validation score   (accuracy)
	0.41s	 = Training   runtime


+--------------------------------------------------------+
| Configuration for experiment     NeuralNetTorch        |
+--------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator |
| Scheduler                        FIFOScheduler         |
| Number of trials                 10                    |
+--------------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20250226_175600/models/NeuralNetTorch



Fitted model: NeuralNetTorch/f8d39_00000 ...
	0.9048	 = Validation score   (accuracy)
	12.47s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/f8d39_00001 ...
	0.3333	 = Validation score   (accuracy)
	12.54s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch/f8d39_00002 ...
	0.6667	 = Validation score   (accuracy)
	5.55s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/f8d39_00003 ...
	0.4286	 = Validation score   (accuracy)
	5.55s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/f8d39_00004 ...
	0.381	 = Validation score   (accuracy)
	5.04s	 = Training   runtime
	0.02s	 = Validation runtime
Fitted model: NeuralNetTorch/f8d39_00005 ...
	0.7619	 = Validation score   (accuracy)
	5.28s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/f8d39_00006 ...
	0.9524	 = Validation score   (accuracy)
	4.81s	 = Training   runtime
	0.0s	 = Validation r

...

Fitted model: LightGBM/T1 ...
	0.9524	 = Validation score   (accuracy)
	0.3s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T2 ...
	0.9524	 = Validation score   (accuracy)
	0.33s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T3 ...
	0.9524	 = Validation score   (accuracy)
	0.29s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T4 ...
	0.8095	 = Validation score   (accuracy)
	0.32s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T5 ...
	0.9524	 = Validation score   (accuracy)
	0.28s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T6 ...
	0.9524	 = Validation score   (accuracy)
	0.28s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: LightGBM/T7 ...
	0.9524	 = Validation score   (accuracy)
	0.23s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBM/T8 ...
	0.3333	 = Validation score   (accuracy)
	0.22s	 = Training   runtime
	

+---------------------------------------------------+
| Configuration for experiment     NeuralNetTorch   |
+---------------------------------------------------+
| Search algorithm                 SearchGenerator  |
| Scheduler                        FIFOScheduler    |
| Number of trials                 10               |
+---------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20250226_175732/models/NeuralNetTorch


Fitted model: NeuralNetTorch/e2875c5f ...
	0.9048	 = Validation score   (accuracy)
	4.7s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch/f7cd08a3 ...
	0.9524	 = Validation score   (accuracy)
	7.98s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/940b3991 ...
	0.8095	 = Validation score   (accuracy)
	5.07s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/2653ccf3 ...
	0.2857	 = Validation score   (accuracy)
	7.78s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/accc99fc ...
	0.2381	 = Validation score   (accuracy)
	5.76s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/dff00feb ...
	0.3333	 = Validation score   (accuracy)
	5.87s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/242523f3 ...
	0.9048	 = Validation score   (accuracy)
	7.71s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: N

Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'LightGBM/T1': 1.0}
	0.9524	 = Validation score   (accuracy)
	0.03s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 67.33s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 1578.4 rows/s (21 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250226_175732")


                  Method  Accuracy  F1-Score
0          Random Search  0.866667  0.866667
1  Bayesian Optimization  0.866667  0.866667


**Grid search**

In [ ]:
from autogluon.tabular import TabularPredictor
from sklearn.metrics import f1_score

# Define hyperparameter grid for Grid Search
hyperparameter_grid = {
    'GBM': [  # Gradient Boosting Machine
        {'learning_rate': 1e-3, 'num_boost_round': 1},  # Epochs = 1
        {'learning_rate': 1e-3, 'num_boost_round': 3},  # Epochs = 3
        {'learning_rate': 1e-3, 'num_boost_round': 5},  # Epochs = 5
        {'learning_rate': 1e-5, 'num_boost_round': 1},  # Epochs = 1
        {'learning_rate': 1e-5, 'num_boost_round': 3},  # Epochs = 3
        {'learning_rate': 1e-5, 'num_boost_round': 5},  # Epochs = 5
    ],
    'CAT': [  # CatBoost
        {'learning_rate': 1e-3, 'iterations': 1},  # Epochs = 1
        {'learning_rate': 1e-3, 'iterations': 3},  # Epochs = 3
        {'learning_rate': 1e-3, 'iterations': 5},  # Epochs = 5
        {'learning_rate': 1e-5, 'iterations': 1},  # Epochs = 1
        {'learning_rate': 1e-5, 'iterations': 3},  # Epochs = 3
        {'learning_rate': 1e-5, 'iterations': 5},  # Epochs = 5
    ],
}

# Perform Grid Search
grid_results = []
for model_type, params_list in hyperparameter_grid.items():
    for params in params_list:
        print(f"Training {model_type} with params: {params}")
        predictor = TabularPredictor(label='label').fit(
            train_data=X_train_df,
            hyperparameters={model_type: params},
            time_limit=300,  # Time limit for each model
        )

        # Evaluate the model
        y_pred = predictor.predict(X_test_df)  # Get predictions
        y_true = X_test_df['label']  # True labels

        # Compute accuracy and F1-score
        accuracy = predictor.evaluate(X_test_df)['accuracy']
        f1 = f1_score(y_true, y_pred, average='weighted')  # Weighted F1-score for multi-class

        # Store results
        grid_results.append({
            'model_type': model_type,
            'params': params,
            'accuracy': accuracy,
            'f1': f1,
        })

# Print Grid Search results
for result in grid_results:
    print(result)

No path specified. Models will be saved in: "AutogluonModels/ag-20250226_175859"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.51 GB / 12.67 GB (82.9%)
Disk Space Avail:   76.04 GB / 112.64 GB (67.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions 

Training GBM with params: {'learning_rate': 0.001, 'num_boost_round': 1}


	0.9524	 = Validation score   (accuracy)
	0.35s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 299.90s of the 299.53s of remaining time.
	Ensemble Weights: {'LightGBM': 1.0}
	0.9524	 = Validation score   (accuracy)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.51s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 5797.4 rows/s (21 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250226_175859")
No path specified. Models will be saved in: "AutogluonModels/ag-20250226_175859-001"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.5

Training GBM with params: {'learning_rate': 0.001, 'num_boost_round': 3}


	0.9524	 = Validation score   (accuracy)
	0.24s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 299.95s of the 299.70s of remaining time.
	Ensemble Weights: {'LightGBM': 1.0}
	0.9524	 = Validation score   (accuracy)
	0.0s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 0.34s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 16977.7 rows/s (21 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250226_175859-001")
No path specified. Models will be saved in: "AutogluonModels/ag-20250226_175900"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.

...

Training CAT with params: {'learning_rate': 1e-05, 'iterations': 1}


No path specified. Models will be saved in: "AutogluonModels/ag-20250226_175902-003"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.52 GB / 12.67 GB (83.0%)
Disk Space Avail:   76.04 GB / 112.64 GB (67.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competiti

Training CAT with params: {'learning_rate': 1e-05, 'iterations': 3}


No path specified. Models will be saved in: "AutogluonModels/ag-20250226_175902-004"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.52 GB / 12.67 GB (83.0%)
Disk Space Avail:   76.04 GB / 112.64 GB (67.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competiti

Training CAT with params: {'learning_rate': 1e-05, 'iterations': 5}
{'model_type': 'GBM', 'params': {'learning_rate': 0.001, 'num_boost_round': 1}, 'accuracy': 0.8666666666666667, 'f1': 0.8666666666666667}
{'model_type': 'GBM', 'params': {'learning_rate': 0.001, 'num_boost_round': 3}, 'accuracy': 0.8666666666666667, 'f1': 0.8666666666666667}
{'model_type': 'GBM', 'params': {'learning_rate': 0.001, 'num_boost_round': 5}, 'accuracy': 0.8666666666666667, 'f1': 0.8666666666666667}
{'model_type': 'GBM', 'params': {'learning_rate': 1e-05, 'num_boost_round': 1}, 'accuracy': 0.8666666666666667, 'f1': 0.8666666666666667}
{'model_type': 'GBM', 'params': {'learning_rate': 1e-05, 'num_boost_round': 3}, 'accuracy': 0.8666666666666667, 'f1': 0.8666666666666667}
{'model_type': 'GBM', 'params': {'learning_rate': 1e-05, 'num_boost_round': 5}, 'accuracy': 0.8666666666666667, 'f1': 0.8666666666666667}
{'model_type': 'CAT', 'params': {'learning_rate': 0.001, 'iterations': 1}, 'accuracy': 1.0, 'f1': 1.0}
{

**Hyperband + Bayesian Optimization hyperparameter**

In [ ]:
from autogluon.tabular import TabularPredictor
from sklearn.metrics import f1_score

# Enable Hyperband + Bayesian Optimization
predictor = TabularPredictor(label='label').fit(
    train_data=X_train_df,
    hyperparameter_tune_kwargs='auto',  # AutoGluon will use Hyperband + Bayesian Optimization
    time_limit=600,  # Total time limit for all models
)

# Evaluate the best model
y_pred = predictor.predict(X_test_df)  # Get predictions
y_true = X_test_df['label']  # True labels

# Compute accuracy and F1-score
accuracy = predictor.evaluate(X_test_df)['accuracy']
f1 = f1_score(y_true, y_pred, average='weighted')  # Weighted F1-score for multi-class

# Print results
print("Best Model Performance:")
print(f"Accuracy: {accuracy}")
print(f"F1-Score: {f1}")

No path specified. Models will be saved in: "AutogluonModels/ag-20250226_175911"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.49 GB / 12.67 GB (82.8%)
Disk Space Avail:   76.04 GB / 112.64 GB (67.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='experimental' : New in v1.2: Pre-trained foundation model + parallel fits. The absolute best accuracy without consideration for inference speed. Does not support GPU.
	presets='best'         : Maximize accuracy. Recommended for most users. Use in competitions 

+----------------------------------------------------+
| Configuration for experiment     NeuralNetFastAI   |
+----------------------------------------------------+
| Search algorithm                 SearchGenerator   |
| Scheduler                        FIFOScheduler     |
| Number of trials                 1000              |
+----------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20250226_175911/models/NeuralNetFastAI


Fitted model: NeuralNetFastAI/b5129e78 ...
	1.0	 = Validation score   (accuracy)
	6.76s	 = Training   runtime
	0.03s	 = Validation runtime
Fitted model: NeuralNetFastAI/649ca93d ...
	0.9524	 = Validation score   (accuracy)
	5.68s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetFastAI/a74c3ae3 ...
	0.9524	 = Validation score   (accuracy)
	4.74s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetFastAI/da8b71a0 ...
	1.0	 = Validation score   (accuracy)
	8.03s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetFastAI/1e66aa1e ...
	1.0	 = Validation score   (accuracy)
	5.11s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetFastAI/09a7604b ...
	1.0	 = Validation score   (accuracy)
	6.41s	 = Training   runtime
	0.02s	 = Validation runtime
Hyperparameter tuning model: LightGBMXT ... Tuning model for up to 41.53s of the 556.99s of remaining time.


	Ran out of time, early stopping on iteration 1. Best iteration is:
	[1]	valid_set's multi_error: 0.0952381
	Stopping HPO to satisfy time limit...
Fitted model: LightGBMXT/T1 ...
	1.0	 = Validation score   (accuracy)
	0.27s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBMXT/T2 ...
	0.9524	 = Validation score   (accuracy)
	0.26s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBMXT/T3 ...
	0.3333	 = Validation score   (accuracy)
	0.23s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBMXT/T4 ...
	0.9524	 = Validation score   (accuracy)
	0.28s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBMXT/T5 ...
	0.3333	 = Validation score   (accuracy)
	0.22s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBMXT/T6 ...
	1.0	 = Validation score   (accuracy)
	0.24s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: LightGBMXT/T7 ...
	0.3333	 = Validation score   (accuracy)
	0.22s

+---------------------------------------------------+
| Configuration for experiment     NeuralNetTorch   |
+---------------------------------------------------+
| Search algorithm                 SearchGenerator  |
| Scheduler                        FIFOScheduler    |
| Number of trials                 1000             |
+---------------------------------------------------+

View detailed results here: /content/AutogluonModels/ag-20250226_175911/models/NeuralNetTorch


Fitted model: NeuralNetTorch/456fb3af ...
	0.9524	 = Validation score   (accuracy)
	5.4s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch/ce6e38ae ...
	1.0	 = Validation score   (accuracy)
	5.86s	 = Training   runtime
	0.0s	 = Validation runtime
Fitted model: NeuralNetTorch/04a4f1d6 ...
	1.0	 = Validation score   (accuracy)
	6.94s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/a1c9ddd0 ...
	0.9524	 = Validation score   (accuracy)
	6.42s	 = Training   runtime
	0.01s	 = Validation runtime
Fitted model: NeuralNetTorch/edc09295 ...
	0.9524	 = Validation score   (accuracy)
	5.52s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBMLarge ... Training model for up to 41.53s of the 335.72s of remaining time.


	0.9524	 = Validation score   (accuracy)
	0.61s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 328.60s of remaining time.
	Ensemble Weights: {'NeuralNetTorch/ce6e38ae': 1.0}
	1.0	 = Validation score   (accuracy)
	0.13s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 271.84s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 3820.3 rows/s (21 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20250226_175911")


Best Model Performance:
Accuracy: 0.9333333333333333
F1-Score: 0.9326599326599326


In [ ]:
import pandas as pd
from tabulate import tabulate

# Data dictionary
data = {
    "Method": [
        "Random Search", "Bayesian Optimization",
        "GBM (lr=0.001, rounds=1)", "GBM (lr=0.001, rounds=3)", "GBM (lr=0.001, rounds=5)",
        "GBM (lr=1e-05, rounds=1)", "GBM (lr=1e-05, rounds=3)", "GBM (lr=1e-05, rounds=5)",
        "CAT (lr=0.001, iterations=1)", "CAT (lr=0.001, iterations=3)", "CAT (lr=0.001, iterations=5)",
        "CAT (lr=1e-05, iterations=1)", "CAT (lr=1e-05, iterations=3)", "CAT (lr=1e-05, iterations=5)",
        "Hyperband + Bayesian Opt."
    ],
    "Accuracy": [
        0.866667, 0.866667,
        0.866667, 0.866667, 0.866667,
        0.866667, 0.866667, 0.866667,
        1.0, 0.966667, 0.966667,
        1.0, 0.966667, 0.966667,
       0.9333333333333333
    ],
    "F1-Score": [
        0.866667, 0.866667,
        0.866667, 0.866667, 0.866667,
        0.866667, 0.866667, 0.866667,
        1.0, 0.966583, 0.966583,
        1.0, 0.966583, 0.966583,
         0.9326599326599326

    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Display the table using tabulate for better readability
print(tabulate(df, headers="keys", tablefmt="grid"))

# Save to CSV for GitHub-friendly format
df.to_csv("model_comparison.csv", index=False)


+----+------------------------------+------------+------------+
|    | Method                       |   Accuracy |   F1-Score |
+====+==============================+============+============+
|  0 | Random Search                |   0.866667 |   0.866667 |
+----+------------------------------+------------+------------+
|  1 | Bayesian Optimization        |   0.866667 |   0.866667 |
+----+------------------------------+------------+------------+
|  2 | GBM (lr=0.001, rounds=1)     |   0.866667 |   0.866667 |
+----+------------------------------+------------+------------+
|  3 | GBM (lr=0.001, rounds=3)     |   0.866667 |   0.866667 |
+----+------------------------------+------------+------------+
|  4 | GBM (lr=0.001, rounds=5)     |   0.866667 |   0.866667 |
+----+------------------------------+------------+------------+
|  5 | GBM (lr=1e-05, rounds=1)     |   0.866667 |   0.866667 |
+----+------------------------------+------------+------------+
|  6 | GBM (lr=1e-05, rounds=3)     |   

Comparison of Manual Tuning vs. Automated Search:

Manual Tuning requires domain expertise and is time-consuming,

Automated Search on the other hand like Grid Search, Random Search, Bayesian Optimization explores a more broader space efficiently.

Automated methods often find better configurations faster, especially for complex models with many hyperparameters.

Automated search is generally preferred for large-scale problems, but combining it with expert insights often yields the best results.